# Outline

- Who needs a priority queue? Emergency departments, data compression algorithms, etc.
- How they work? By organizing data so that the highest (or lowest) valued element is the next one to return.

- Binary trees as arrays
- Explain basic structure using note with 2 children
- Demonstrate but do not code traversal of a tree
- Discuss insertion in binary search tree
- Look at a complete tree and ask if there is a simpler way to represent
- Discuss how node content can be mapped in array (ok, list)
- Develop index functions for left/right children and parent for each list[position]
- Pivot discussion to a priority queue -- describe functionality
- Introduce [`@dataclass`](https://docs.python.org/3/library/dataclasses.html) decorator
- Suggest $\mathcal O (n)$ immplementation with linear search that swaps [0] and [most_important_idx]
- Conceptualize $\mathcal O(\log n)$ approach with binary tree.
- Map binary tree to array
- Operationalize at array level

## Example

Waiting room of an ER, in order of arrivals:

`[broken angle, stroke, sprained wrist, kidney stone]`

Reorganized as a priority queue:

`[stroke, broken angle, sprained wrist, kidney stone]`

After the first case (`stroke`) is taken, the list will be reorganized with the most severe case moved to the front.

`[kidney stone, broken angle, sprained wrist]`

Notice that the list is _not_ sorted; just the most important item, based on some metric, is placed at the front. The only requirement here is that `list[0]` is always the most important item. The remaining elements can be randomly organized as far as we are concerned.

## Naive priority queue

- Linear scan to find the most importnat element and bring it to the front.
- Implement with integers before writing a generic class.
- (Time permitting): discussion about generics.


In [ ]:
class SimplePriorityQ:

    _DEFAULT_CAPACITY: int = 10

    def __init__(self, capacity: int = _DEFAULT_CAPACITY) -> None:
        """Initialize an empty priority queue with given capacity."""
        self._capacity: int = capacity
        # Initialize a list at the specified capacity. This way we
        # can place values directly to the list instead of appending,
        # treating it like an actual array.
        self._underlying: list[int] = [None] * self._capacity
        # Tracks how many elements are in the "array". The invariant
        # here is that 0 ≤ size ≤ capacity.
        self._size: int = 0

    def insert(self, value: int) -> None:
        """Insert value into the priority queue."""
        if self._size >= self._capacity:
            raise Exception("Priority queue is full")
        self._underlying[self._size] = value
        self._size += 1
        self._move_important_to_front()

    def _move_important_to_front(self):
        # Where is the most important element?
        max_idx = self._most_important_idx()
        # Swap positions, bringign the most imporant element to the front
        self._swap(0, max_idx)

    def _most_important_idx(self):
        """Finds and returns the position of the most important
        element in the underlying array"""
        max_idx = 0
        for i in range(1, self._size):
            if self._underlying[i] > self._underlying[max_idx]:
                max_idx = i
        return max_idx

    def _swap(self, position, with_position):
        """Swaps positions between two elements in the list. The code uses
        the basic three-variable trick instead of the Pythonic trick
        a,b = b,a
        for better illustration and portability in other languages.
        """
        temp = self._underlying[position]
        self._underlying[position] = self._underlying[with_position]
        self._underlying[with_position] = temp

    def remove_most_important(self) -> int:
        """Remove and return the maximum element from the priority queue."""
        if self.is_empty():
            raise Exception("Priority queue is empty")
        # Grab the first element, so that we can return it
        most_important = self._underlying[0]
        # Use a temporary array that is just 1 element shorter to
        # copy everything underlying array element except for the
        # first one.
        temp_list = [None] * (len(self._underlying) - 1)
        for i in range(1, (self._size)):
            temp_list[i - 1] = self._underlying[i]
        # Replace the underlying array with the shorter temp array
        self._underlying = temp_list
        # Adjust the size
        self._size -= 1
        # Make sure that the most important element in this shorter
        # underlynig array is now at its front
        self._move_important_to_front()
        # Done; return the removed element to the user.
        return most_important

    def is_empty(self) -> bool:
        """Return True if the priority queue is empty, False otherwise."""
        return (self._size) == 0

    def __bool__(self) -> bool:
        return self.is_empty()

    # String representation constants

    _Q_EMPTY = "Queue is empty."
    _Q_IN_QUEUE = f"in queue: ["
    _Q_CONTENTS_CLOSING = f" ]"
    _Q_CONTENTS_DELIMITER = f", "
    _Q_SINGULAR = f"element"
    _Q_PLURAL = f"{_Q_SINGULAR}s"

    def __str__(self) -> str:
        # Initialize return string to assume empty queue
        text = f"{self._Q_EMPTY}"
        # If queue is not empty prepare to populate return string with data
        if self._size > 0:
            # Prepare singular or plural noun, 1 element, 2 elements etc
            elements = self._Q_SINGULAR if self._size == 1 else self._Q_PLURAL
            text = f"{self._size} {elements} {self._Q_IN_QUEUE} {self._underlying[0]}"
            for i in range(1, self._size):
                text += f"{self._Q_CONTENTS_DELIMITER}{self._underlying[i]}"
            text += f"{self._Q_CONTENTS_CLOSING}"
        return text

The code above can be tested, quickly, with the following.

```python
q = SimplePriorityQ()
q.insert(1); q.insert(3); q.insert(0); q.insert(4)
print(q); print(q.remove_max()) # expect [ 4, 1, 0, 3 ],  4
print(q); print(q.remove_max()) # expect [ 3, 0, 1 ],     3
print(q); print(q.remove_max()) # expext [ 1, 0 ],        1
print(q); print(q.remove_max()) # expect [ 0 ],           0
print(q); print(q.remove_max()) # expect `Queue is empty.` then exception
```


## Improvements over naive priority queue

The naive priority queue runs in $\mathcal O(n)$ time. This is not bad, but it can be done faster.

Consider the data structure below.

![](./images/MaxHeapTree.jpg)

The data in it seem all over the place but there are two notable properties. The largest value node is at the top of the structure. And each node is greater than or equal to both its immediate two nodes below it. This arrangement is called the **maximum heap property** and can be formalized as $p \geq\max\left(c_L, c_R\right)$ for the value of a node $p$ and the two nodes $c_L$ and $c_R$ immediately under it as shown below.

![](./images/parent_child.jpg)

We use the terms _parent,_ _left child,_ and _right child_ to describe this arrangement.

If either child violates the requirement $p \geq\max\left(c_L, c_R\right)$, then we must swap the offeding value with the value in the parent node. For example, both children below are greater than the parent node:

![](./images/101212med.jpg)

After finding the largest of the two children and swaping with the value of the parent node, the requirement $p \geq\max\left(c_L, c_R\right)$ is again restored.

![](./images/121110med.jpg)

Verifying and restoring the property $p \geq\max\left(c_L, c_R\right)$ is quite easy. (`Node` objects are defined as a _dataclass_ for simplicity, below).


In [ ]:
@dataclass
class Node:
    value: int
    left: "Node" = None
    right: "Node" = None


def validate(p: Node) -> bool:
    """Check if the node satisfies the heap property."""
    return (
        p is not None
        and (p.left is not None and p.left.value <= p.value)
        and (p.right is not None and p.right.value <= p.value)
    )


def restore(p: Node) -> Node:
    """Restore the heap property at node p by swapping values as needed."""
    # Find the largest of the children
    if p.left is not None:
        # Proceed only if there is at least one child. If there is only one child,
        # it must be the left child. Assign largest to left child initially.
        largest: Node = p.left
        if p.right is not None and p.right.value > largest.value:
            # If there is a right child, and it is larger than the left child,
            # assign largest to right child instead.
            largest = p.right
        # If the largest child is larger than p, swap their values.
        if largest.value > p.value:
            largest.value, p.value = p.value, largest.value
    return p

In addition to the *maximum heap property* we can also arrange data according to the **minimum heap property:**
$$
p \leq\min\left(c_L, c_R\right)
$$

## Implement as an array

Conceivably, we could assemble a collection of these `Node` objects beginning with one on top, then two children, each with two children etc. The result would be the structure shown earlier and also below.

![](./images/MaxHeapTree.jpg)

If every `Node` has two children except for the nodes in the bottom layer, we can predict the total number of nodes $N$ in a structure with $L$ layers:

$$ 2^{L-1} \leq N \leq 2^L-1 $$

This allows us to implement the tree in a much simpler manner, using an array (or list) with size $2^L-1$. All we need is to be able to tell where, in the array, are the two children of a node and, conversely, where is the parent of each node. This is easy to accomplish after we rearrange the nodes of the tree on a line, where the the top node is first, nodes of the next layer from left to right are next, and so on. The linear arrangement is shown below.

![](./images/linear_arrangement.jpg)

If we use an array to hold these nodes, the parent-children relation can be expressed as simple algebraic functions of the array indices. For example, after we observe a few relations between parent and children indices, we can reach the conclusion shown in the table below.

|   $p$    |  $c_L$   |  $c_R$   |
| :------: | :------: | :------: |
|    0     |    1     |    2     |
|    1     |    3     |    4     |
|    2     |    5     |    7     |
| $\vdots$ | $\vdots$ | $\vdots$ |
|   $i$    |  $2i+1$  |  $2i+2$  |

So we can write the functions that generate the index of left and write children as

$$
\begin{align*}
c_L(p) & = 2p+1 \\
c_R(p) & = 2p+2 \\ & = c_L(p) + 1
\end{align*}
$$

Or, in plain Python

```python
def left_child(parent: int) -> int:
    return 2*parent + 1

def right_child(parent: int) -> int:
    return 2*parent + 2
```

Conversely, we can find the parent of every node using the inverse relation:

$$
p(c) = \left\lfloor \dfrac{c+1}{2}\right\rfloor
$$

and in plain Python:

```python
def parent(child: int) -> int:
    return (child + 1) // 2
```

With these relations in place, we can write functions `violation_occurs` and `restore` from earlier as array operations. In the code below, we assume that the methods have access to a the `underlying` list with the linear arrangement of the tree. Method `violation_occurs` is the boolean complement of earlier method `validate`.

In [ ]:

def violation_occurs(parent: int) -> bool:
    """Check whether the heap property is violated at the given parent index."""
    return (
        underlying[left_child(parent)] > underlying[parent]
        and underlying[right_child(parent)] > underlying[parent]
    )

def restore(parent: int) -> None:
    """Restore the heap property at the given parent index."""
    # Find the largest of the children
    left_idx = left_child(parent)
    if underlying[left_idx] is not None:
        # Proceed only if there is at least one child. If there is only one child,
        # it must be the left child. Assign largest to left child initially.
        largest: int = left_idx
        right_idx: int = right_child(parent)
        if underlying[right_idx] is not None and underlying[right_idx] > underlying[left_idx]:
            # If there is a right child, and it is larger than the left child,
            # assign largest to right child instead.
            largest = right_idx
        # Swap
        underlying[parent], underlying[largest] = underlying[largest], underlying[parent]
        return largest # where the parent node ended at


## Fast removal

The main feature of this data structure is that it keeps the most important elemenet in its first position and returns it on demand. As shown in the simple implementation above, `remove_most_important` grabs whatever is at position `[0]` of the `underlying` list. In that simple implementation we look for the next important item in the remaining list, to place it to the front, so that we'll be ready for the next request.

While it takes only one step to return the most important element among $n$ elements (because its location is known), it takes up to $n-1$ steps to find the next important element and bring it to the front.

There is a way to do this faster, in no more than $\log_2n$ steps. This is illustrated below.

![](./images/maxheap_removal.jpg)

In the structure above, the most important element is on the top. When we request it, the structure removes and returns it, and replaces it with its last element as shown in part (a). Then, beginning from the top node that just got updated, we check if meets the other feature of this data structure, that a parent node $p$ is at least as important as both its children $c_L$ and $c_R$. In this example, important is measured by greater value, i.e., $p \geq \max\left(c_L,c_R\right)$.

Everytime $p < \min\left(c_L, c_R\right)$, we swap the parent with the largest of the two children and repeat the process until the property is fully restored. This is shown in parts (b), (c), and (d) above. There will be up to $L-1$ swaps, where $L$ is the number of layers. In a data structure with $n$ elements there are $\left\lceil \log_2n\right\rceil$ layers.

Given the list-based implementation, the comparisons between parent and children nodes can done with list indices. This is shown in the code below, which makes use of methods sketched out earlier.


In [ ]:
def restore_from_top():
    parent = 0
    while violation_occurs(parent):
        parent = restore(parent)

## Fast addition

<mark>This is a topic in the assignment</mark>

Addition of new data can be modeled on the tree structure. New data are added to the last available position of the tree. In the example below, we assume that the last node on the bottom layer was vacant, and we inserted a new value. 

![](./images/maxheap_addition.jpg)

We then proceed to compare the newly placed node with its parent. If the heap property is violated (i.e., `violation_occurs` returns a `True`) we swap and repeat until the property is restored.
